Drawing on Zandbergen's (2011) "Error propagation modeling for terrain analysis...", this script simulates the influence of elevation measurement errors on the calculated volumes of landslides. This analysis is a component of the broader research presented in "Ramos, C. and Arima, E. 2025 Multi-decadal sediment dynamics of a tropical watershed - the relative roles of surface erosion and shallow landsliding. Earth Surface Processes and Landforms.

Implemented using arcpy library

Written by: E. Arima: University of Texas at Austin

Date: February 2025

In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *
import pandas as pd
import os

In [ ]:
#Set path to your folder
path = r""
env.workspace = path
#Overwrite output
env.overwriteOutput = True
os.chdir(path)

In [ ]:
#Read the DEM raster files to variables (elevations only where landslides were identified)
dem2018 = r"2018_dem_yah_jan25.tif"

dem2016 = r"2016_dem_yah_jan25.tif"
#set this raster as the extent
env.extent = dem2016
env.cellSize = dem2016

In [ ]:
#Read the landslides polygon file into a variable
lndslides = r"digitizedlandslides_yah_final_Edited.shp"
#set environment mask
#env.mask = lndslides

In [ ]:
#Set number of simulations
nsim = 251

In [ ]:
#Set RMSEs
#Non-vegetated vertical accuracy for 2016 is 9.4 cm, according to https://www.fisheries.noaa.gov/inport/item/55314
rmse2016 = 0.094
#Non-vegetated vertical accuracy for 2018 is 6.83 cm, according to https://www.fisheries.noaa.gov/inport/item/60105
rmse2018 = 0.0683
#Distribution parameter for random error: normal distribution with mean zero, sd = 1
pdist = "NORMAL 0.0, 1.0"

In [ ]:
#loop
for s in range(1, nsim):
    #create a random raster, normal distribution with mean zero, sd = rmse
    #output raster
    outrnd2016 = f"rnd2016{s}.tif"
    arcpy.management.CreateRandomRaster(path, outrnd2016, pdist, cellsize = 1)
    #Create autocorrelated error raster
    fs2016 = FocalStatistics(outrnd2016, NbrCircle(5, "CELL"), "MEAN")
    #outfs2016 = f"fs2016{s}.tif"
    #fs2016.save(outfs2016)
    #Obtain standard deviation of fs raster
    stobj2016 = arcpy.GetRasterProperties_management(fs2016, "STD")
    std2016 = float(stobj2016.getOutput(0))
    print(f"The standard deviation of 2016 simulation {s} is: {std2016}")
    #Bring fs to sd = 1
    fsdiv2016 = Raster(fs2016)/std2016
    #
    #Multiply by RMSE to bring it to sd = rmse
    fsmult2016 = Raster(fsdiv2016)*rmse2016
    #Add error to DEM
    demfs2016 = Raster(dem2016) + Raster(fsmult2016)
    #outdem2016 = f"demfs2016_{s}.tif"
    #demfs2016.save(outdem2016)
    #
    #repeat process for 2018 DEM
    #output raster
    outrnd2018 = f"rnd2018{s}.tif"
    arcpy.management.CreateRandomRaster(path, outrnd2018, pdist, cellsize = 1)
    #Create autocorrelated error raster
    fs2018 = FocalStatistics(outrnd2018, NbrCircle(5, "CELL"), "MEAN")
    #outfs2018 = f"fs2018{s}.tif"
    #fs2018.save(outfs2018)
    #Obtain standard deviation of fs raster
    stobj2018 = arcpy.GetRasterProperties_management(fs2018, "STD")
    std2018 = float(stobj2018.getOutput(0))
    print(f"The standard deviation of 2018 simulation {s} is: {std2018}")
    #Bring fs back to sd = 1
    fsdiv2018 = Raster(fs2018)/std2018
    #
    #Multiply by RMSE to bring it to sd = rmse
    fsmult2018 = Raster(fsdiv2018)*rmse2018
    #Add error to DEM
    demfs2018 = Raster(dem2018) + Raster(fsmult2018)
    #outdem2018 = f"demfs2018_{s}.tif"
    #demfs2018.save(outdem2018)
    #
    #Subtract 2018 - 2016
    deminus = Raster(demfs2018) - Raster(demfs2016)
    #outminus= f"deminus{s}.tif"
    #deminus.save(outminus)
    #Calculate zonal statistics as table
    # by ID number
    outTable = f"zonalstat{s}.dbf"
    outzonal = ZonalStatisticsAsTable(lndslides, "ID", deminus, outTable, "DATA", "ALL")
    #
    #Clean up, delete tif files
    #
    #
    toDel = [outrnd2016, outrnd2018]
    arcpy.management.Delete(toDel)
    print(f"Simulation {s} completed.")
print("********************************")
print("All simulations completed.")